In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

RuntimeError: Could not initialize camera.  Please see error trace.

In [4]:
detections = model(camera.value)

NameError: name 'camera' is not defined

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets
detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

In [ ]:
from jetbot import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
image_number = 0
object_number = 0

print(detections[image_number][object_number])

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
from jetbot import bgr8_to_jpeg
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

    
def norm(vec):
    """计算2D向量长度"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """找出离图像中心最近的目标"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
       
    

import numpy as np

def detection_center(detection):
    """计算目标的xy中心坐标"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

def dist(v0, v1):
    return np.dot(v0, v1) / np.sqrt(np.dot(v0, v0)) / np.sqrt(np.dot(v1, v1))

def execute(change):
    image = change['new']
        
    # 计算所有被检测到的目标
    detections = detectModel(image)
    
#     x0, y0, x1, y1 = 200, 0, 200, 10
    x0, y0, x1, y1 = change['line']
    
    w, h  = image.shape[0], image.shape[1]
#     print(w, h)
#     cv2.line(image, (x0, y0), (x1, y1), (255,0,0), 10)
    x0 /= w
    y0 /= h
    x1 /= w
    y1 /= h
    
    closest_detection = None
    for det in detections[0]:
        if det['label'] == 1:
            continue;
        bbox = det['bbox']
#         cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)

        center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
        center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
#         print(center_x, center_y)
        dis =  dist(np.array([x1-x0, y1-y0]), np.array([center_x - x0, center_y - y0]))
        if closest_detection is None:
            closest_detection = det
        else:
            
            xx, yy = detection_center(closest_detection)
            if dis < dist(np.array([x1-x0, y1-y0]), np.array([xx - x0, yy - y0])):
                closest_detection = det
        
    if closest_detection is None:
        label_widget.value = 1
#         print("------")
    else:
        label_widget.value = closest_detection['label']
        
        bbox = closest_detection['bbox']
#         cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
        
    # 更新图像小部件
#     image_widget.value = bgr8_to_jpeg(image)
# execute({'new': camera.value})

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

In [ ]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()